In [2]:
from dotenv import load_dotenv
import os
from neo4j_helpers import *


load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
app = BasicNeo4j(DB_HOST, DB_USER, DB_PASSWORD)

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")


Failed to write data to connection ResolvedIPv4Address(('127.0.0.1', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cpu",
)
messages = [
	{"role": "user", "content": "Who are you? Please, answer in pirate-speak."},
]
prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
print(outputs[0]["generated_text"][len(prompt):])

Ahoy, matey! I be the scurvy-stricken buccaneer known as the Dread Pirate Dreadbeard! I walk the plank o' the seven seas, huntin' for the gold and glory that be mine. Avast me mateys, I'm a riddle wrapped in a riddle, a kraken with a twist! I sail the stormy seas with a kraken's heart and a pirate's grin!


In [3]:
default_creation_query="""CREATE (runner:Runner {name: "Kelvin Kiptum Cheruiyot", nationality: "Kenyan", birth_date: date("1999-12-02"), death_date: date("2024-02-11")})
CREATE (record:Record {event: "marathon", time: "less than 2:02", record_type: "course record"})
CREATE (runner)-[:HOLDS_RECORD]->(record)
CREATE (achievement:Achievement {description: "the only person in history to run the marathon in less than two hours and one minute in a record-eligible race"})
CREATE (runner)-[:ACHIEVED]->(achievement)
CREATE (race1:Race {name: "Valencia Marathon 2022", date: date("2022-10-23"), time: "2:02", rank: "third"})
CREATE (runner)-[:PARTICIPATED_IN]->(race1)
CREATE (race2:Race {name: "London Marathon 2023", date: date("2023-04-23"), time: "2:01:25", rank: "second"})
CREATE (runner)-[:PARTICIPATED_IN]->(race2)
CREATE (race3:Race {name: "Chicago Marathon 2023", date: date("2023-10-23"), time: "2:00:35", rank: "first"})
CREATE (runner)-[:PARTICIPATED_IN]->(race3)
CREATE (ranking:Ranking {rank: 1, year: 2024, event: "men's marathon"})
CREATE (runner)-[:RANKED_FIRST_IN]->(ranking)
"""
app.erase_database()
app.execute_custom(cypher_query=default_creation_query)

'Data insertion successful.'

In [3]:
tools_lib = Neo4jTools(neo4j_app=app)
print(tools_lib.find_all_labels())

Runner, Record, Achievement, Race, Ranking


In [4]:
tools_lib = Neo4jTools(neo4j_app=app)
print(tools_lib.find_all_relationship_types())

HOLDS_RECORD, ACHIEVED, PARTICIPATED_IN, RANKED_FIRST_IN


In [4]:
from langchain.tools import Tool

tools_lib = Neo4jTools(neo4j_app=app)
tools = [
    Tool.from_function(
        func=tools_lib.find_all_relationship_types,
        name="Search for existing relationship types",
        description="Must be used before creation a cypher in order to get all existing relationship types."
    ),
    Tool.from_function(
        func=tools_lib.find_all_labels,
        name="Search for existing node labels",
        description="Must be used before creation a cypher in order to get all existing node labels."
    ),
    Tool.from_function(
        func=tools_lib.execute_query,
        name="Execute query in database",
        description="Accepts Cypher-formatted query and executes it in database."
    )
]

In [9]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import Ollama

llm = Ollama(model="gemma")
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

agent.run(
    "What races Kelvin Kiptum Cheruiyot participated in?"
)



> Entering new AgentExecutor chain...
**Question:** What races Kelvin Kiptum Cheruiyot participated in?

**Thought:** I need to find out what races Kelvin Kiptum Cheruiyot has participated in.

**Action:**

```
{
  "action": "Search for relationship types",
  "action_input": "Kelvin Kiptum Cheruiyot"
}
```

**
Observation: HOLDS_RECORD, ACHIEVED, PARTICIPATED_IN, RANKED_FIRST_IN
Thought:## Answer

**Question:** What races Kelvin Kiptum Cheruiyot participated in?

**Thought:** I need to find out what races Kelvin Kiptum Cheruiyot has participated in.

**Action:**

```
{
  "action": "Search for relationship types",
  "action_input": "Kelvin Kiptum Cheruiyot"
}
```

**
Observation: HOLDS_RECORD, ACHIEVED, PARTICIPATED_IN, RANKED_FIRST_IN
Thought:## Answer

**Question:** What races Kelvin Kiptum Cheruiyot participated in?

**Thought:** I need to find out what races Kelvin Kiptum Cheruiyot has participated in.

**Action:**

```
{
  "action": "Search for relationship types",
  "action_inpu

KeyboardInterrupt: 

In [24]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

agent.run(
    "What races Kelvin Kiptum Cheruiyot participated?"
)



> Entering new AgentExecutor chain...
Question: What races Kelvin Kiptum Cheruiyot participated in?
Thought: We need to first find the node label for Kelvin Kiptum Cheruiyot and then see the relationships to the races.
Action:
```
{
  "action": "Search for node labels",
  "action_input": ""
}
```

Observation: Runner, Record, Achievement, Race, Ranking
Thought:```
{
  "action": "Execute query in database",
  "action_input": "MATCH (r:Runner {name: 'Kelvin Kiptum Cheruiyot'})-[:PARTICIPATED_IN]->(race) RETURN race"
}
```
Observation: [{'race': {'date': neo4j.time.Date(2022, 10, 23), 'name': 'Valencia Marathon 2022', 'rank': 'third', 'time': '2:02'}}, {'race': {'date': neo4j.time.Date(2023, 4, 23), 'name': 'London Marathon 2023', 'rank': 'second', 'time': '2:01:25'}}, {'race': {'date': neo4j.time.Date(2023, 10, 23), 'name': 'Chicago Marathon 2023', 'rank': 'first', 'time': '2:00:35'}}]
Thought:Could not parse LLM output: Final Answer
Observation: Invalid or incomplete response
Thought:

'Kelvin Kiptum Cheruiyot participated in the Valencia Marathon 2022, London Marathon 2023, and Chicago Marathon 2023.'

In [5]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

agent.run(
    "I am building graph database with knowledge. There is already information in the database. I want you parse this data below and put it into the existing database. The data to parse: \n Eliud Kipchoge EGH (born 5 November 1984) is a Kenyan long-distance runner who competes in the marathon and formerly specialized in the 5000 metres. Regarded as one of the greatest marathon runners of all time, he is the 2016 and 2020 Olympic marathon champion, and was the world record holder in the marathon from 2018 to 2023, with a time of 2:01:09 set at the 2022 Berlin Marathon, until that record was broken by Kelvin Kiptum at the 2023 Chicago Marathon with a time of 2:00:35. He has run five of the 10 fastest marathons in history."
)

/Users/es/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/Users/es/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Question: How can we parse the information about Eliud Kipchoge into the existing database?
Thought: We need to extract relevant information such as Eliud Kipchoge's name, nationality, date of birth, achievements, records, and notable races to create nodes and relationships in the database.
Action:
```
{
  "action": "Execute query in database",
  "action_input": "CREATE (runner:Eliud {name: 'Eliud Kipchoge', nationality: 'Kenyan', date_of_birth: '5 November 1984'})"
}
```

Observation: Data insertion successful.
Thought:Now that we have inserted basic information about Eliud Kipchoge into the database, we should continue adding more nodes and relationships based on the additional details provided in the text.
Action:
```
{
  "action": "Execute query in database",
  "action_input": "MATCH (runner:Eliud) SET runner.specialization = 'marathon', runner.former_specialization = '5000 metres', runner.greatest = true, runner.olympic_champion = [2016, 202

'Information about Eliud Kipchoge has been successfully parsed into the existing database with nodes representing him as a runner, his achievements, records, and notable races, along with appropriate relationships between them.'